In [ ]:
# pip install pygenomeviz

In [1]:
from pygenomeviz import GenomeViz
import random
import sys

# random.seed(5)

In [2]:
# process minimap alignment
home="/Users/luorunpeng/Downloads/all-e/Research/project-haplotype-phasing/ctgContact-dup"
chrom_file=home+"/chrom_id_pst104e.lst"

ref_file="/Users/luorunpeng/Downloads/all-e/Research/project-haplotype-phasing/run_pst104e/reference/chrAll.fna"



In [3]:
# parse chrom file
chr_id2name = {}
name2chr_id = dict()
lefts = []
rights = []
with open(chrom_file, "r") as chrom_fd:
    for line in chrom_fd:
        chrom_id, left, right = line.strip().split("\t")
        chr_id2name[chrom_id] = (left, right)
        name2chr_id[left] = chrom_fd
        name2chr_id[right] = chrom_fd
        lefts.append(left)
        rights.append(right)
    chrom_fd.close()

def get_strand(strand: str):
    if strand == "+":
        return 1
    else:
        return -1

def parse_aln_file(aln_file: str):
    chrom_dict = {}
    for name in name2chr_id.keys():
        chrom_dict[name] = []
    with open(aln_file, "r") as paf_fd:
        for line in paf_fd:
            # perfect alignment
            splited = line.strip().split("\t")
            seg_no = str(splited[0])
            strand = get_strand(str(splited[4]))
            r_name = str(splited[5])
            r_len = int(splited[6])
            r_start, r_end = int(splited[7]), int(splited[8])
            mapq = int(splited[11])
            chrom_dict[r_name].append((seg_no, strand, mapq, r_start, r_end))
        paf_fd.close()
    return chrom_dict

In [4]:
def get_rlens(ref_file: str):
    res = {}
    with open(ref_file, "r") as fa_fd:
        sid = ""
        seq = ""
        for line in fa_fd:
            if line.startswith(">"):
                # process previous entry
                if sid != "":
                    res[sid] = len(seq)
                sid = line.strip()[1:].split()[0]
                seq = ""
            else:
                seq += line.strip()
        fa_fd.close()
        if sid != "":
            res[sid] = len(seq)
    return res

# parse the Quast csv file
def parse_tsv_file(tsv_file: str):
    chrom_dict = {}
    for name in name2chr_id.keys():
        chrom_dict[name] = []
    with open(tsv_file, "r") as tsv_fd:
        tsv_fd.readline()
        i = 0
        for line in tsv_fd.readlines():
            if i == 0:
                # line 0
                if "unaligned" in line:
                    continue
                splited = line.strip().split()
                s1, e1 = int(splited[0]), int(splited[1])
                rname = str(splited[4])
                cname = str(splited[5])
                mrate = float(splited[6])
                chrom_dict[rname].append((cname, -2, mrate, s1, e1))
            else:
                # line 1
                pass # contig alignment type
            
            i = (i+1) % 2
        tsv_fd.close()
    return chrom_dict


In [14]:
def draw_track(pts: list, track, strand, fg_color, add_label):
    for cds in pts:
        (seg_no, _, mapq, r_start, r_end) = cds
        # strand = 1 # ignore the strand when plot
        plotstyle = "box"
        # random.choice(plotstyles)
        color = fg_color

        # label=seg_no, labelsize=10, 
        if add_label:
            track.add_feature(r_start, r_end, strand, facecolor=color, plotstyle=plotstyle, label=seg_no, labelsize=10, labelrotation=90, linewidth=0.5)
        else:
            track.add_feature(r_start, r_end, strand, facecolor=color, plotstyle=plotstyle, linewidth=0.5)
        # track.add_feature(start, end, strand, label=plotstyle, labelsize=20, facecolor=color, plotstyle=plotstyle, labelvpos="top", labelhpos="center", labelrotation=60, linewidth=1)
    return

def draw_phasing_plot(tsv_file1: str, tsv_file2: str, ref_file: str, output):
    gv = GenomeViz()

    chrom_dict1 = parse_tsv_file(tsv_file1)
    chrom_dict2 = parse_tsv_file(tsv_file2)
    chrom_len = get_rlens(ref_file)

    plotstyles = ("bigarrow", "arrow", "bigbox", "box", "bigrbox", "rbox")
    colors = ("orange", "blue", "lime", "red")

    color1="blue"
    color2="red"

    # gv = GenomeViz()
    for chr_id, (left, right) in chr_id2name.items():

        # if chr_id != '3':
        #     continue

        left_size, right_size = chrom_len[left], chrom_len[right]
        max_size = max(left_size, right_size)

        # l_track = gv.add_feature_track(name, max_size)
        l_track = gv.add_feature_track(f"{chr_id}A\n{chr_id}B", max_size)
        l_track.add_feature(0, max_size, 1, facecolor="grey", plotstyle="box", linewidth=1)
        
        draw_track(chrom_dict1[left], l_track, 1, color1, False)
        draw_track(chrom_dict1[right], l_track, 1, color2, False)

        l_track.add_feature(0, max_size, -1, facecolor="grey", plotstyle="box", linewidth=1)

        draw_track(chrom_dict2[right], l_track, -1, color2, False)
        draw_track(chrom_dict2[left], l_track, -1, color1, False)


    # fig = gv.plotfig(dpi=300)
    gv.savefig(output, dpi=300)
    return


In [17]:
quast_base="/Users/luorunpeng/Downloads/all-e/Research/project-haplotype-phasing/quast_analysis"
quast_dir_all=quast_base+"/quast_pst104e_all/contigs_reports"
quast_dir_hapA=quast_base+"/quast_pst104e_hapA/contigs_reports"
quast_dir_hapB=quast_base+"/quast_pst104e_hapB/contigs_reports"

# hifiasm (hic)
h1_hifiasm_hic_tsv=quast_dir_hapA+"/all_alignments_pst104e_ul_hic-asm-hic-hap2-p_ctg.tsv"
h2_hifiasm_hic_tsv=quast_dir_hapB+"/all_alignments_pst104e_ul_hic-asm-hic-hap1-p_ctg.tsv"

draw_phasing_plot(h1_hifiasm_hic_tsv, h2_hifiasm_hic_tsv, ref_file, home+"/imgs/pst104e_hifiasm_hic_phasing_plot.png")

# hifiasm + greenhill
h1_hifiasm_greenhill_tsv=quast_dir_hapA+"/all_alignments_greenhill_hifiasm_H2.tsv"
h2_hifiasm_greenhill_tsv=quast_dir_hapB+"/all_alignments_greenhill_hifiasm_H1.tsv"

draw_phasing_plot(h1_hifiasm_greenhill_tsv, h2_hifiasm_greenhill_tsv, ref_file, home+"/imgs/pst104e_hifiasm_greenhill_phasing_plot.png")

# hifiasm + algorithm
h1_hifiasm_tree_tsv=quast_dir_hapA+"/all_alignments_dir_phasing_hifiasm_pst104e_tree_bin1.tsv"
h2_hifiasm_tree_tsv=quast_dir_hapB+"/all_alignments_dir_phasing_hifiasm_pst104e_tree_bin0.tsv"

draw_phasing_plot(h1_hifiasm_tree_tsv, h2_hifiasm_tree_tsv, ref_file, home+"/imgs/pst104e_tree_hifiasm_phasing_plot.png")

In [ ]:
quast_base="/Users/luorunpeng/Downloads/all-e/Research/project-haplotype-phasing/quast_analysis"
quast_dir_all=quast_base+"/quast_pst104e_all/contigs_reports"
quast_dir_hapA=quast_base+"/quast_pst104e_hapA/contigs_reports"
quast_dir_hapB=quast_base+"/quast_pst104e_hapB/contigs_reports"

# verkko (hic)
h1_verkko_hic_tsv=quast_dir_hapA+"/all_alignments_assembly-haplotype1.tsv"
h2_verkko_hic_tsv=quast_dir_hapB+"/all_alignments_assembly-haplotype2.tsv"

draw_phasing_plot(h1_verkko_hic_tsv, h2_verkko_hic_tsv, ref_file, home+"/imgs/pst104e_verkko_hic_phasing_plot.png")



# verkko + greenhill
h1_verkko_greenhill_tsv=quast_dir_hapA+"/all_alignments_greenhill_verkko_H1.tsv"
h2_verkko_greenhill_tsv=quast_dir_hapB+"/all_alignments_greenhill_verkko_H2.tsv"

draw_phasing_plot(h1_verkko_greenhill_tsv, h2_verkko_greenhill_tsv, ref_file, home+"/imgs/pst104e_verkko_greenhill_phasing_plot.png")



# verkko + algorithm
h1_verkko_tree_tsv=quast_dir_hapA+"/all_alignments_dir_phasing_verkko_pst104e_tree_bin1.tsv"
h2_verkko_tree_tsv=quast_dir_hapB+"/all_alignments_dir_phasing_verkko_pst104e_tree_bin0.tsv"

draw_phasing_plot(h1_verkko_tree_tsv, h2_verkko_tree_tsv, ref_file, home+"/imgs/pst104e_tree_verkko_phasing_plot.png")

In [ ]:
draw_phasing_plot(h1_hifiasm_hic_tsv, h2_hifiasm_hic_tsv, ref_file, home+"/pt76_hifiasm_hic_phasing_plot.png", "Hifiasm (HiC)")
draw_phasing_plot(h1_hifiasm_greenhill_tsv, h2_hifiasm_greenhill_tsv, ref_file, home+"/pt76_hifiasm_greenhill_phasing_plot.png", "Hifiasm + GreenHill")
draw_phasing_plot(h1_tree_hifiasm_hic_tsv, h2_tree_hifiasm_hic_tsv, ref_file, home+"/pt76_tree_hifiasm_hic_phasing_plot.png", "Hifiasm+Algorithm")
draw_phasing_plot(h1_verkko_hic_tsv, h2_verkko_hic_tsv, ref_file, home+"/pt76_verkko_phasing_plot.png", "Verkko (HiC)")
draw_phasing_plot(h1_verkko_greenhill_tsv, h2_verkko_greenhill_tsv, ref_file, home+"/pt76_verkko_greenhill_phasing_plot.png", "Verkko + GreenHill")
draw_phasing_plot(h1_tree_verkko_tsv, h2_tree_verkko_tsv, ref_file, home+"/pt76_tree_verkko_phasing_plot.png", "Verkko + Algorithm")

gv.savefig("pt76_chr3_phasing_plot.png", dpi=300)